# ***CS5040: Linear Optimization***

#### *Assignment-3*

Team Members:

1.   Aaditya Kehdekar- EE19BTECH11027
2.   Akshay Magre- EE19BTECH11002
3.   Megavath Praveen- AI19BTECH11018
4.   Tushar More- EE19BTECH11025





Question 3.

*   We are given a Matrix A of order m*n, vector c of length n, vector b of length m and some feasible point z of length n.
*   We have to take into consideration the following assumptions.
    * Rank of A is n.
*   Unlike previous problems we have no knowledge of degeneracy and also the initial feasible point is not provided.

*   Our task here remains to check for the degeneracy of the polytope.
*   Another task would be to find an appropriate initial feasible point.

# Get initial feasible point

In [ ]:
import numpy as np
eta = 1e-10

def get_feasible_point(A, b, c):
    if np.all((b >= 0)):     # if every element in b are positive,
        return np.zeros(c.shape)      # then origin is a feasible point
    else:
        for i in range(50):
            m = A.shape[0]
            n = A.shape[1]
            rand_i = np.random.choice(m, n)  # consider any random n of m constraints
            randA_i = A[rand_i]
            randb_i = b[rand_i]
            try:
                # Find Equality Solution for this n constraints
                invA_i = np.linalg.inv(randA_i)
                possible_X = invA_i @randb_i

                # If the calculated X is satisfying all of the constraint
                if (np.all((A@possible_X - b <= 0))):
                    return possible_X
                else:
                    continue
            # If the considered random constraints matrix is not invertible, pass
            except:
                pass

# Check Degeneracy and if present then make it non-degenerate

In [ ]:

def check_degeneracy(A, b, c):
    # Get any feasible point for the given set of constraints
    X = get_feasible_point(A, b, c)

    # Find number of rows satisfied by X with equality
    equality_idx = np.where(np.abs(np.dot(A, X)-b) < eta)[0]

    # If number of rows is not equal to number of variables, It is degenerate
    if len(equality_idx) == A.shape[1]:
      print("Non-degenerate")
      return 0
    print("Degenerate")    # Degenerate = no unique solution
    return 1


def make_non_degenerate(A, b, c):

    # consider the last m - n rows of b vector
    rows_to_be_modified = A.shape[0]-A.shape[1]

    num_iter = 0
    # get unique feasible point(point satisfying only the first n independent rows), by disturbing AX = b, equality
    while True:
        if(num_iter < 1000):
            num_iter += 1

            # add a small random value to each of these rows
            bt = np.empty_like(b,dtype='float64')
            bt[:] = b
            bt[:rows_to_be_modified] += np.random.uniform(eta, eta*10, size=rows_to_be_modified)
        else:
            # add a small random value to each of these rows
            bt = b
            bt[:rows_to_be_modified] += np.random.uniform(0.1, 10, size=rows_to_be_modified)
            
        # If it becomes Non-degenerate then exit
        if not check_degeneracy(A,bt, c):
            break
    return A, bt, c

# Get Optimal Vertex

In [ ]:
def move_to_vertex(A, b, c, X):
    # Get direction vectors of vertex X
    equality_idx = np.where(np.abs(A@X-b) < eta)[0]
    A_bar = A[equality_idx]  # A' = Matrix of n linearly independent rows, which are satisfied by X with equality

    # Find Z = Matrix having direction vectors as columns
    Z = -np.linalg.inv(A_bar.T)

    # Find costs through these directions
    costs = Z@c

    # Find Directions with positive costs
    p_cost_dir = np.where(costs > 0)[0]

    # If there are no more positive cost directions available
    if len(p_cost_dir) == 0:
        return None      # the present vertex is optimal
    else:
        # consider positive cost direction vector
        v = Z[p_cost_dir[0]]

        # If it is unbounded in the present direction
        if len(np.where(A@v > 0)[0]) == 0:
            print('It is Unbounded')
            exit()

        equality_idx = np.where(np.abs(A@X-b) < eta)[0]
        n_equality_idx = ~np.isin(np.arange(len(A)), equality_idx)
        n_equal_A = A[n_equality_idx]        # n_A = Matrix of rows other than satisfied by X with equality
        n_equal_b = b[n_equality_idx]        # n_b = corresponding b values of above rows

        # Maximum t in feasible neighbour(X + tv)
        n = n_equal_b - n_equal_A@X
        d = n_equal_A@v
        n = n[np.where(d > 0)[0]]
        d = d[np.where(d > 0)[0]]
        s = n/d
        t = np.min(s[s >= 0])

        # Return the maximum feasible neighbour of X
        return X + t*v

def get_optimal_vertex(A, b, c, X):
    while True:
        # Find neighbour with greater cost
        Z = move_to_vertex(A, b, c, X)

        # If the neighbour isn't available
        if Z is None:       
            break        # the present vertex is the optimal
        else:
            X = Z         # else move to neighbour
    return X

**Test 1**

In [ ]:

A = np.array([ [-1, 0], [0, -1], [-1, -1], [0, 1]])
b = np.array([-1, -1, -4, 3])
c = np.array([-2, -3])

# Check Degeneracy
s = check_degeneracy(A, b, c)

# change Input to non degenerate
A, b, c = make_non_degenerate(A, b, c)

# Find the initial feasible point
X = get_feasible_point(A, b, c)
print('Initial Feasible Point: ', X)

# Find optimal solution
X = get_optimal_vertex(A, b, c, X)
print('The Optimal Vertex is :', X)
print('The Maximum Value is: ', np.dot(c, X))

Degenerate
Non-degenerate
Initial Feasible Point:  [1. 3.]
The Optimal Vertex is : [3. 1.]
The Maximum Value is:  -8.999999999640995


Here we can see that for test case 1, It was degenerate, so we converted it into non-degenerate and found out optimal vertex.

**Test 2**

In [ ]:
A = np.array([[-1, -1], [1, 1], [-1,0], [0,-1]])
b = np.array([-1, 2, 0 ,0])
c = np.array([5, 2])

# Check Degeneracy
s = check_degeneracy(A, b, c)

# Find the initial feasible point
X = get_feasible_point(A, b, c)
print('Initial Feasible Point: ', X)

# Find optimal solution
X = get_optimal_vertex(A, b, c, X)
print('The Optimal Vertex is :', X)
print('The Maximum Value is: ', np.dot(c, X))

Non-degenerate
Initial Feasible Point:  [2. 0.]
The Optimal Vertex is : [2. 0.]
The Maximum Value is:  10.0


Here we can see that for test case 2, It was already non-degenerate, so we did not converted it into non-degenerate
, instead directly found optimal vertex.Also hence we got initial feasible point as optimal vertex.

**Test 3**

In [ ]:
A = np.array([[-1, -1], [-1,0], [0,-1]])
b = np.array([-4, -1, -1])
c = np.array([-5, -2])

# Check Degeneracy
s = check_degeneracy(A, b, c)

# Find the initial feasible point
X = get_feasible_point(A, b, c)
print('Initial Feasible Point: ', X)

# Find optimal solution
X = get_optimal_vertex(A, b, c, X)
print('The Optimal Vertex is :', X)
print('The Maximum Value is: ', np.dot(c, X))

Non-degenerate
Initial Feasible Point:  [1. 3.]
The Optimal Vertex is : [1. 3.]
The Maximum Value is:  -11.0


Here too, we can see that for test case 3, It was already non-degenerate, so we directly found optimal vertex.Also hence we got initial feasible point as optimal vertex.